# Dynamisches Labyrinth

## Aufgabenstellung

Ihr Ziel ist es, durch ein Labyrinth aus 20 Feldern zu navigieren. Sie können entweder einen Spielstein schieben oder Ihre Spielfigur entlang des begehbaren Wegs bewegen. Das Schieben eines Steins verschiebt eine ganze Zeile und bewegt den Stein auf die gegenüberliegende Seite des Spielfelds. Wenn Ihre Figur auf das herausgeschobene Feld gelangt, wird sie auf das hineingeschobene Feld gesetzt. Sie müssen mit möglichst wenigen Zügen von der Eingangsposition zu den Ausgangspunkten (rote Pfeile) gelangen, wobei der Ein- und Ausgang begehbar sein muss.

## Theoretische Vorüberlegungen

### Einige Fragen
- Beim A* Algorithmus: Wenn kein Weg existiert, wie wird dann der beste Knoten (best_node) ausgewählt, und ist der Algorithmus in solchen Fällen noch "einfach" anwendbar?
- Sind Anpassungen an der Heuristik notwendig, um bessere Ergebnisse in einem dynamischen Labyrinth zu erzielen, das sich mit jeder Aktion verändert?
- Wird zuerst der Spielstein verschoben oder wird zuerst der Spieler an die optimale Position bewegt, um den besten Pfad zu finden?
- Existieren bestimmte Spielsteine, die für einen möglichst optimalen Pfad besser geeignet sind als andere, und wenn ja, welche Eigenschaften machen sie besser für die Navigation im Labyrinth?

## Limits des entwickelten Algorithmus
- ...

## Vorgehensweise zur Lösung des Rätsels im dynamischen Labyrinth
1. Einlesen des Spielfelds, um die Ausgangsbedingungen zu erfassen.
2. Kodierung der Spielsteine und Definition der Grid-Klasse zur effizienten Verwaltung des Spielfelds.
3. Anpassung des A* Algorithmus:
    - Implementierung des A* Algorithmus für die Pfadfindung.
    - Anpassung der Heuristik, um die Entscheidungen basierend auf dem "bestmöglichen Spielstein" zu verbessern.
    - Implementierung der Logik zum Verschieben von Reihen/Spielern im Labyrinth.
4. Erstellung eines grafischen Prototyps für die visuelle Darstellung des Spielverlaufs und der Lösungswege.

### 1. Einlesen des Spielfelds
- Das Spielfeld wird als n x m Matrix aus einer CSV-Datei geladen.
- Die CSV-Datei enthält Werte im Bereich von 0 bis 9.
- Jeder Wert repräsentiert einen bestimmten Spielstein, weitere Details folgen im nächsten Schritt.
- Der freie Spielstein liegt in der linken unteren Ecke der CSV-Datei und wird z.B. als "`2;;;`" dargestellt.

In [1]:
import csv

csv_path = './data/Puzzle_3.csv'

spielfeld = []

with open(csv_path, 'r') as csv_datei:
    csv_reader = csv.reader(csv_datei, delimiter=";")
    for zeile in csv_reader:
        spielfeld.append(zeile)

## 2. Spielsteine und Grid
### 2.1 Spielsteine definieren
Erklärung: Es gibt 10 verschiedene Spielsteine die angeben, in welche Richtung der Spieler sich bewegen und nicht bewegen kann. Dabei werden die verschiedenen Spielsteine in einer
- Matrix von 0 - 9 dargestellt und
- jeder Spielstein erhält zudem eine 2x2 Matrix der die möglichen Richtungen angibt. 

Dabei gilt folgendes: [ left, top, right, bottom] wobei eine 1 für eine Verbindung und eine 0 für keine Verbindung steht zum nächsten Feld steht. Als Beipsiel, [0, 0, 1, 1] steht für [-, -, right, bottom], also ist dieser Spielstein eine Ecke die eine Verbindung von unten nach rechts besitzt.

In [2]:
# Kodierung der Spielsteine 
# [ left, top, right, bottom]
tile0 = [0, 0, 1, 1]
tile1 = [1, 0, 0, 1]
tile2 = [0, 1, 1, 0]
tile3 = [1, 1, 0, 0]
tile4 = [1, 0, 1, 1]
tile5 = [1, 1, 0, 1]
tile6 = [1, 1, 1, 0]
tile7 = [0, 1, 1, 1]
tile8 = [0, 1, 0, 1]
tile9 = [1, 0, 1, 0]

In [3]:
# Spielsteine und Kodierungen
tiles = [tile0, tile1, tile2, tile3, tile4, tile5, tile6, tile7, tile8, tile9]

## 2.2 Grid definieren
Die Klasse `Grid` verwaltet das Spielfeld für den A*-Algorithmus und die Spielsteinbewegungen. Sie initialisiert das Spielfeld, ermöglicht das Abrufen und Setzen von Knotenwerten sowie das Verschieben des Spielers und der Spielsteine. Die Methode `get_adjacent` ermittelt benachbarte Knoten, die durch begehbare Wege verbunden sind.

In [4]:
class Grid:
    '''
    Das Grid erleichtert das Arbeiten im A* Stern, verschieben der
    Spielsteine (Reihen) und des Spielers.
    '''
    def __init__(self, src, dst, matrix):
        '''
        Definiert Start und Ziel um Grid.
        Initialisiert alle Knoten und Knoten Werte

        Parameters: src, dst
        Returns: none
        '''
        self.src = src
        self.dst = dst
        self.off_tile = int(matrix[-1][0])
        self.rows = len(matrix)-1
        self.cols = len(matrix[0]) if self.rows > 0 else 0
        self.nodes = [(i, j) for i in range(self.rows) for j in range(self.cols)]
        self.grid_values = [[0 for _ in range(self.cols)] for _ in range(self.rows)]

        for i in range(self.rows):
            for j in range(self.cols):
                self.grid_values[i][j] = int(matrix[i][j])

    def get_value(self, row, col):
        '''
        Wert der zur Kodierung der Spielsteine verwendet wird 

        Parameters: row, col
        Returns: int
        '''
        if 0 <= row < self.rows and 0 <= col < self.cols:
            return self.grid_values[row][col]
        return None
    
    def set_value(self, row: int, col: int, new_value: int(0-9)):
        '''
        Spielstein Kodierungs Wert (0-9) um z.B. Spielsteine verschieben

        Parameters: row, col, new_value
        Returns: none
        '''
        self.grid_values[row][col] = int(new_value)

    def move_player_left(self, player_src):
        '''
        Verschiebt den Spieler nach links und gibt die neue Position zurück

        Parameters: player_src
        Returns: new_position
        '''
        row, col = player_src

        if col == 0:
            new_position = (row, self.cols - 1)
        else:
            new_position = (row, (col - 1) % self.cols)

        return new_position
    
    def move_row_left(self, row_index, new_tile):
        '''
        Verschiebt die gegebene Reihe (index) nach links, 
        setzt ein neuen Spielstein (new_tile) rechts ein und 
        gibt den entfernten linken Spielstein zurück

        Parameters: row_index, new_tile
        Returns: moved_out
        '''
        row = self.grid_values[row_index]
        moved_out = row[0]

        for i in range(1, self.cols):
            self.set_value(row_index, i - 1, row[i])
        self.set_value(row_index, self.cols - 1, new_tile)

        return moved_out
        
    def move_player_right(self, player_src):
        '''
        Verschiebt den Spieler nach rechts und gibt die neue Position zurück

        Parameters: player_src
        Returns: new_position
        '''
        row, col = player_src
        if col == self.cols - 1:
            new_position = (row, 0)
        else:
            new_position = (row, (col + 1) % self.cols)

        return new_position
    
    def move_row_right(self, row_index, new_tile):
        '''
        Verschiebt die gegebene Reihe (index) nach rechts, 
        setzt ein neuen Spielstein (new_tile) links ein und 
        gibt den entfernten rechten Spielstein zurück

        Parameters: row_index, new_tile
        Returns: moved_out
        '''
        row = self.grid_values[row_index]
        moved_out = row[-1]
        for i in range(self.cols - 1, 0, -1):
            self.set_value(row_index, i, row[i-1])
        self.set_value(row_index, 0, new_tile)

        return moved_out

    def get_nodes(self):
        '''
        Parameters: None
        Returns: nodes
        '''       
        return self.nodes
    
    def get_adjacent(self, node):   
        '''
        Ermittelt alle horizontal und vertikal liegenden Nachbarn eines Knotens.
        Gibt alle benachbarten Knoten zurück.

        Parameters: node
        Returns: adjacent_nodes
        '''     
        y, x = node
        adjacent_nodes = []

        # Horizontal Nachbarn
        for dx in [-1, 1]:
            new_x, new_y = x + dx, y
            # Wenn im Grid dann
            if 0 <= new_y < self.rows and 0 <= new_x < self.cols:
                # linker Nachbar
                if dx == -1:
                    possible_path_new_x = tiles[self.get_value(new_y, new_x)]
                    possible_path_x = tiles[self.get_value(y, x)]

                    # Sind die Nachbarn mit einem Weg verbunden? 
                    if possible_path_new_x[2] == 1 and possible_path_x[0] == 1:                   
                        adjacent_nodes.append((new_y, new_x))

                # rechter Nachbar
                if dx == 1:
                    possible_path_new_x = tiles[self.get_value(new_y, new_x)]
                    possible_path_x = tiles[self.get_value(y, x)]

                    # Sind die Nachbarn mit einem Weg verbunden?
                    if possible_path_new_x[0] == 1 and possible_path_x[2] == 1: 
                        adjacent_nodes.append((new_y, new_x))

        # Vertikal Nachbarn
        for dy in [-1, 1]:
            new_x, new_y = x, y + dy

            # Wenn im Grid dann
            if 0 <= new_y < self.rows and 0 <= new_x <= self.cols:
                # Oberer Nachbar 
                if dy == -1:
                    possible_path_new_y = tiles[self.get_value(new_y, new_x)]
                    possible_path_y = tiles[self.get_value(y, x)]

                    # Sind die Nachbarn mit einem Weg verbunden? 
                    if possible_path_new_y[3] == 1 and possible_path_y[1] == 1:
                        adjacent_nodes.append((new_y, new_x))

                # Unterer Nachbar
                if dy == 1:
                    possible_path_new_y = tiles[self.get_value(new_y, new_x)]
                    possible_path_y = tiles[self.get_value(y, x)]
                    # Sind die Nachbarn mit einem Weg verbunden? 
                    if possible_path_new_y[1] == 1 and possible_path_y[3] == 1:
                        adjacent_nodes.append((new_y, new_x))
                

        return adjacent_nodes

## Spielfeld Grid initialiseren aus spielfeld csv

In [5]:
spielfeld_grid = Grid((4,0), (0,3), spielfeld)

## 3. A* anhand lösbarem Beispiel

Some text

In [6]:
# Replace this with your actual heuristic function
def heuristic_cost_estimate(current, goal, grid):
    # Example: Man
    x1, y1 = current
    x2, y2 = goal

    # Wert aus dem Grid mit x1, x2
    grid_tile_number = grid.get_value(x1,y1)
    
    # Tile bestimmen durch Wert aus Grid
    grid_tile_value = tiles[grid_tile_number]
    # Wenn Tile mit Weg nach oben +0 (Wird besser gewichtet)
    if grid_tile_value[1]:
        return abs(x2 - x1) + abs(y2 - y1)
        #return (((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5)
    
    # Wenn Tile nicht mit Weg nach oben +2
    return abs(x2 - x1) + abs(y2 - y1) + 2
    #return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5 + 1

Zwei verschiedene Möglichkeiten das Grid zu verschieben (Also ein Spielstein von außen ins spielfeld schieben):
- Erstens durch Regression 
- Zweitens durch verschachtelte While Schleife

In [7]:
def a_star(grid):
    nodes = grid.get_nodes()
    closed_set = []
    prev = {node: None for node in nodes}
    g_score = {node: float("inf") for node in nodes}
    src = grid.src # das ist eigentlich unser Spieler
    dst = grid.dst
    g_score[src] = 0
    f_score = {node: float("inf") for node in nodes}
    f_score[src] = heuristic_cost_estimate(src, dst, grid)
    
    
    best_node_cost = f_score[src]
    best_node = grid.src
    open_set = [best_node]

    in_game = False

    # best_node ist auch grid.src
    while True:
        # wenn spieler am start dann nach unten offenes tile
        if tiles[grid.get_value(best_node[0],best_node[1])][3] != 1 and in_game == False:
            #in_game = False
            open_set.remove(best_node)

        # V1 um die while nochmal eine while sollange unser bestnode nicht u
        while open_set:
            u = min(open_set, key=f_score.get)
            open_set.remove(u)
            closed_set.append(u)
            
            # Ziel gefunden
            if u == dst:
                print('Found end')
                break

            else:
                for v in grid.get_adjacent(u):
                    new_g = g_score[u] + 1
                    
                    if v not in open_set and v not in closed_set:
                        open_set.append(v)
                        g_score[v] = new_g
                        f_score[v] = new_g + heuristic_cost_estimate(v, dst, grid)
                        # merke den knoten wo du stehtst (3, 0)
                        
                        # besten knoten im "subtree"
                        # wir wählen den erst Besten ("<") aufgrund der weiter benötigten Schritte (wenn kein Ende vorhanden)
                        if heuristic_cost_estimate(v, dst, grid) < best_node_cost:
                            best_node = v
                            best_node_cost = heuristic_cost_estimate(v, dst, grid)
                                       
                        prev[v] = u
                    elif ((v in open_set) or (v in closed_set)) and (new_g < g_score[v]):
                        g_score[v] = new_g
                        f_score[v] = new_g + heuristic_cost_estimate(v, dst, grid)
                        prev[v] = u
                        
                        if v in closed_set:
                            open_set.append(v)
                            closed_set.remove(v)

                        # besten knoten im "subtree"
                        # wir wählen den erst Besten ("<") aufgrund der weiter benötigten Schritte (wenn kein Ende vorhanden)
                        if heuristic_cost_estimate(v, dst, grid) < best_node_cost:
                            best_node = v
                            best_node_cost = heuristic_cost_estimate(v, dst, grid)
                    
        # Ziel gefunden
        if best_node == dst and tiles[grid.get_value(dst[0],dst[1])][1] == 1:      
            print(tiles[grid.get_value(dst[0],dst[1])][1])
            print('Found end')
            break

        prev_best_node = best_node

        if in_game:
            # Alle geraden Reihen dann nach rechts
            if (best_node[0] % 2 == 0):
                grid.off_tile = grid.move_row_right(best_node[0], grid.off_tile)
                best_node = grid.move_player_right(best_node) 
            else: 
                grid.off_tile = grid.move_row_left(best_node[0], grid.off_tile)
                best_node = grid.move_player_left(best_node)

            prev[best_node] = prev_best_node
        else:
            if (best_node[0] % 2 == 0):
                grid.off_tile = grid.move_row_right(best_node[0], grid.off_tile)
            else: 
                grid.off_tile = grid.move_row_left(best_node[0], grid.off_tile)
        
        if tiles[grid.get_value(best_node[0],best_node[1])][3] == 1 and in_game == False:
            in_game = True
        
        open_set = [best_node]

        if best_node == dst and (tiles[grid.get_value(best_node[0],best_node[1])][1] == 1):
            print("Found End")
            break  
    
    
    # Now reconstruct the path if it was found
    path = []
    if prev[dst] is not None:
        current = dst
        while current is not None:
            path.append(current)
            current = prev[current]
        path = path[::-1]  # Reverse to get the path from src to dst
    
    return path


- Start nach unten: wenn kein Spielstein nach unten dann reihe drüber überprüfen ob ein Stein mit nach unten und diesen dann herausschieben und vewenden als Start "Stein"
- Reihe in der der Spieler steht: wenn kein Spielstein nach oben, dann in reihe drüber prüfen ob stein nach oben, wenn ja dann Stein herausschieben und vewenden
- Reihe über dem Spieler: wenn kein Spielstein nach unten, dann in anderer Reihe 

In [8]:

optimal_path = a_star(spielfeld_grid)

Found end
1
Found end


## Grafische Ausgabe

In [9]:
string_map = {
    0: "┌",
    1: "┐",
    2: "└",
    3: "┘",
    4: "┬",
    5: "┤",
    6: "┴",
    7: "├",
    8: "|",
    9: "-",
}

In [10]:
def create_path_matrix(path, rows, cols, grid: Grid):
    optimal_path_matrix = [[' ' for _ in range(cols)] for _ in range(rows)]

    for node in path:
        y, x = node
        optimal_path_matrix[y][x] = string_map[grid.get_value(y,x)]  # Mark the cell in the path

    return optimal_path_matrix

path_matrix = create_path_matrix(optimal_path, spielfeld_grid.rows, spielfeld_grid.rows, spielfeld_grid)

# Gib die Matrix aus
for row in path_matrix:
    print(' '.join(row))


  |      
  ┤      
  ├      
  ┤      
┌ ┘ └ ┘  


In [11]:
# Create a matrix of strings
matrix = [[int(num) if num != '' else None for num in row] for row in spielfeld[:-1]]

# Display the matrix
for row in matrix:
    print(row)

string_matrix = [[string_map[num] for num in row] for row in matrix]

print("---")
# Display the string matrix
for row in string_matrix:
    print(" ".join(row))

[2, 8, 4, 7]
[1, 5, 6, 1]
[0, 7, 2, 9]
[6, 5, 5, 9]
[3, 4, 0, 3]
---
└ | ┬ ├
┐ ┤ ┴ ┐
┌ ├ └ -
┴ ┤ ┤ -
┘ ┬ ┌ ┘


3. A* anhand lösbarem Beispiel

- checken grenzen
    - wenn laufen nicht möglich, dann schieben
        
- verschieben  
    - Zeile über Spielfigur in möglich Richtung
    
- checken funktion/algo
- Listen führen: Open/Close

- Heuristik:
    - 1 Schritt = +1
    - 1 Verschieben = +1
    - Heuristischer wert = Schritte bis zum Ziel
        -   ABER wenn nicht Öffnung nach oben +2

WAS TUN BEI GLEICHSTAND?
AB WANN VERSCHIEBEN (auch wenn laufen möglich)

- jeden möglichen weg berechnen
- jeden möglichen weg nach verschiebungen